In [ ]:
%matplotlib inline


In [ ]:
import pickle


def get_calibration_data():
    # Read in the saved camera matrix and distortion coefficients
    # These are the arrays you calculated using cv2.calibrateCamera()
    return pickle.load( open( "./output_images/calibration.p", "rb" ) )


## Writeup Template
### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

---

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"
[image2]: ./test_images/test1.jpg "Road Transformed"
[image3]: ./examples/binary_combo_example.jpg "Binary Example"
[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points
### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---
### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it!

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the first code cell of the IPython notebook located in "./examples/example.ipynb" (or in lines # through # of the file called `some_file.py`).  

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

[//]: # (Image References)

[image1]: ./examples/undistort_output.png "Undistorted"

![alt text][image1]

In [ ]:
# %load calibrate_camera.py
import cv2
from glob import iglob
import matplotlib.pyplot as plt
import numpy as np
import pickle


def get_corners(image, pts_per_row, pts_per_column, flags=None):
    return cv2.findChessboardCorners(image, (pts_per_row, pts_per_column), flags)


def draw_image_corners(image, pts_per_row, pts_per_column, corners, ret):
    cornered = cv2.drawChessboardCorners(image, (pts_per_row, pts_per_column), corners, ret)
    return cornered


def get_calibration_pts(img_files_regex, pts_per_row, pts_per_column, num_channels):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((pts_per_column * pts_per_row, 3), np.float32)
    objp[:,:2] = np.mgrid[0:pts_per_row, 0:pts_per_column].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = iglob(img_files_regex)

    # Step through the list and search for chessboard corners
    for idx, filename in enumerate(images, start=1):
        img = cv2.imread(filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = get_corners(gray, pts_per_row, pts_per_column)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = draw_image_corners(img, pts_per_row, pts_per_column, corners, ret)
            write_name = './test_out/corners_found'+str(idx)+'.jpg'
            cv2.imwrite(write_name, img)

    return objpoints, imgpoints


def undistort_image(image, matrix, distortion_coefficients):
    return cv2.undistort(image, matrix, distortion_coefficients, None, matrix)


if __name__ == '__main__':
    img_files_regex = './camera_cal/calibration*.jpg'
    pts_per_row = 9
    pts_per_column = 6
    num_channels = 3

    objpoints, imgpoints = get_calibration_pts(img_files_regex, pts_per_row, pts_per_column, num_channels)

    # Test undistortion on an image
    img = cv2.imread('./camera_cal/calibration3.jpg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_size = (img.shape[1], img.shape[0])

    # Do camera calibration given object points and image points
    ret, matrix, distortion_coefficients, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Save the camera calibration results
    dist_pickle = {}
    dist_pickle["mtx"] = matrix
    dist_pickle["dist"] = distortion_coefficients
    pickle.dump( dist_pickle, open( "./output_images/calibration.p", "wb" ) )

    # Undistort image and save results to file
    undistorted = undistort_image(img, matrix, distortion_coefficients)
    undistorted_gray = cv2.cvtColor(undistorted, cv2.COLOR_BGR2GRAY)
    ret, corners = get_corners(undistorted_gray, pts_per_row, pts_per_column)
    undistorted = draw_image_corners(undistorted, pts_per_row, pts_per_column, corners, ret)
    cv2.imwrite('test_out/test_undist.jpg', undistorted)

    # Visualize distortion correction
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(undistorted)
    ax2.set_title('Undistorted Image', fontsize=30)


### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.
To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
![alt text][image2]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.
I used a combination of color and gradient thresholds to generate a binary image (thresholding steps at lines # through # in `another_file.py`).  Here's an example of my output for this step.  (note: this is not actually from one of the test images)

![alt text][image3]

## Sobel Operator

In [ ]:
# %load thresholds.py
import cv2
import matplotlib.pyplot as plt
import numpy as np
from calibrate_camera import get_calibration_data, undistort_image
from ipywidgets import interact


# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output


# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx ** 2 + sobely ** 2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag) / 255
    gradmag = (gradmag / scale_factor).astype(np.uint8)
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output


def test_sobel_thresh(min=0, max=0, orient='x'):
    threshed = abs_sobel_thresh(img, orient, min, max)
    print('min: {}  max: {}'.format(min, max))
    #     plt.imshow(threshed, cmap='gray', interpolation='nearest', aspect='auto')
    # Plot the result
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(orig_img)
    ax1.set_title('Original Image', fontsize=24)
    ax2.imshow(img)
    ax2.set_title('Original Image [Undistorted]', fontsize=24)
    ax3.imshow(threshed, cmap='gray')
    ax3.set_title('Thresholded Gradient [Sobel]', fontsize=24)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


def set_orient(p):
    global a, b, o
    o = p
    test_sobel_thresh(a, b, o)


def set_min(n):
    global a, b, o
    a = n
    test_sobel_thresh(a, b, o)


def set_max(x):
    global a, b, o
    b = x
    test_sobel_thresh(a, b, o)


calibration = get_calibration_data()
matrix = calibration['mtx']
distortion_coefficients = calibration['dist']
orig_img = cv2.cvtColor(cv2.imread('./test_images/signs_vehicles_xygrad.png'), cv2.COLOR_BGR2RGB)
img = undistort_image(orig_img, matrix, distortion_coefficients)

#Test Sobel threshold
a = None
b = None
o = 'x'

interact(set_orient, p='x')
interact(set_min, n=(0, 255))
interact(set_max, x=(0, 255))

# # Test Magnitude Threshold
# mag_binary = mag_thresh(img, sobel_kernel=3, mag_thresh=(30, 100))
# # Plot the result
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# f.tight_layout()
# ax1.imshow(img)
# ax1.set_title('Original Image', fontsize=50)
# ax2.imshow(mag_binary, cmap='gray')
# ax2.set_title('Thresholded Magnitude', fontsize=50)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


## Magnitude of the Gradient

In [ ]:
# %load thresholds.py
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact


# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output


# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx ** 2 + sobely ** 2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag) / 255
    gradmag = (gradmag / scale_factor).astype(np.uint8)
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output


def test_sobel_thresh(min=0, max=0, orient='x'):
    threshed = abs_sobel_thresh(img, orient, min, max)
    print('min: {}  max: {}'.format(min, max))
    #     plt.imshow(threshed, cmap='gray', interpolation='nearest', aspect='auto')
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(threshed, cmap='gray')
    ax2.set_title('Thresholded Gradient [Sobel]', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


def set_orient(p):
    global a, b, o
    o = p
    test_sobel_thresh(a, b, o)


def set_min(n):
    global a, b, o
    a = n
    test_sobel_thresh(a, b, o)


def set_max(x):
    global a, b, o
    b = x
    test_sobel_thresh(a, b, o)


img = cv2.cvtColor(cv2.imread('./test_images/signs_vehicles_xygrad.png'), cv2.COLOR_BGR2RGB)

# Test Sobel threshold
# a = None
# b = None
# o = 'x'

# interact(set_orient, p='x')
# interact(set_min, n=(0, 255))
# interact(set_max, x=(0, 255))

# # Test Magnitude Threshold
mag_binary = mag_thresh(img, sobel_kernel=3, mag_thresh=(30, 100))
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(mag_binary, cmap='gray')
ax2.set_title('Thresholded Magnitude', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


### Direction of the Gradient

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle


# Read in an image
image = mpimg.imread('./test_images/signs_vehicles_xygrad.png')

# Define a function that applies Sobel x and y, 
# then computes the direction of the gradient
# and applies a threshold.
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output    

# Run the function
dir_binary = dir_threshold(image, sobel_kernel=15, thresh=(0.7, 1.3))
# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(dir_binary, cmap='gray')
ax2.set_title('Thresholded Grad. Dir.', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Color Threshold

In [ ]:
from calibrate_camera import get_calibration_data, undistort_image

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


# Edit this function to create your own pipeline.
def pipeline(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))

    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    return color_binary, combined_binary
    

calibration = get_calibration_data()
matrix = calibration['mtx']
distortion_coefficients = calibration['dist']

image = mpimg.imread('./test_images/test5.jpg')
undistorted = undistort_image(image, matrix, distortion_coefficients)
color_binary, combined_binary = pipeline(undistorted)

# Plot the result
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
f.tight_layout()

ax1.imshow(image)
ax1.set_title('Original Image', fontsize=24)

ax2.imshow(color_binary)
ax2.set_title('Pipeline Result [Color binary]', fontsize=24)

ax3.imshow(combined_binary, cmap='gray')
ax3.set_title('Pipeline Result [Combined binary]', fontsize=24)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)


# image = mpimg.imread('./test_images/test5.jpg')
# hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
# s_channel = hls[:,:,2]
# thresh = (180, 255)
# binput = s_channel
# gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
# binary = np.zeros_like(binput)
# binary[(binput > thresh[0]) & (binput <= thresh[1])] = 1

# plt.figure(figsize=(15, 25))

# plt.subplot(211)
# plt.axis('off')
# plt.title('Original')
# plt.imshow(image)

# plt.subplot(212)
# plt.axis('off')
# plt.title('S binary')
# plt.imshow(binary, cmap='gray');



In [ ]:
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from glob import glob

top = 450
bottom = 670

p1 = (580, top)
p2 = (250, bottom)
p3 = (700, top)
p4 = (1060, bottom)

images = glob('./test_images/straight_lines*.jpg')

COLUMNS = 1
rows = math.ceil(len(images) / COLUMNS)
plt.figure(figsize=(75, 75))


def display_img(image, idx, item_count, title):
    cv2.line(image, p1, p2, [255, 0, 0], 2)
    cv2.line(image, p3, p4, [255, 0, 0], 2)
    cv2.line(image, p1, p3, [255, 0, 0], 2)
    cv2.line(image, p2, p4, [255, 0, 0], 2)

    plt.subplot(rows, COLUMNS, idx)
    plt.plot(p1[0], p1[1], 'X')
    plt.plot(p2[0], p2[1], 'X')
    plt.plot(p3[0], p3[1], 'X')
    plt.plot(p4[0], p4[1], 'X')
    plt.title(title)
    plt.imshow(image)


# Step through the list and search for chessboard corners
for idx, filename in enumerate(images, 1):
    image = mpimg.imread(filename)
    display_img(image, idx, len(images), filename.split('/')[-1])


#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `warper()`, which appears in lines 1 through 8 in the file `example.py` (output_images/examples/example.py) (or, for example, in the 3rd code cell of the IPython notebook).  The `warper()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```
src = np.float32(
    [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
    [((img_size[0] / 6) - 10), img_size[1]],
    [(img_size[0] * 5 / 6) + 60, img_size[1]],
    [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])
dst = np.float32(
    [[(img_size[0] / 4), 0],
    [(img_size[0] / 4), img_size[1]],
    [(img_size[0] * 3 / 4), img_size[1]],
    [(img_size[0] * 3 / 4), 0]])

```
This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 585, 460      | 320, 0        | 
| 203, 720      | 320, 720      |
| 1127, 720     | 960, 720      |
| 695, 460      | 960, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

![alt text][image4]

In [ ]:
# %load warper.py
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Read in the saved camera matrix and distortion coefficients
# These are the arrays you calculated using cv2.calibrateCamera()
dist_pickle = pickle.load( open( "./output_images/calibration.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Read in an image
img = cv2.imread('./camera_cal/calibration13.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

# MODIFY THIS FUNCTION TO GENERATE OUTPUT
# THAT LOOKS LIKE THE IMAGE ABOVE
def corners_unwarp(img, nx, ny, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
        # If we found corners, draw them! (just for fun)
        cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
        # Choose offset from image corners to plot detected corners
        # This should be chosen to present the result at the proper aspect ratio
        # My choice of 100 pixels is not exact, but close enough for our purpose here
        offset = 175 # offset for dst points
        # Grab the image shape
        img_size = (gray.shape[1], gray.shape[0])
        print('img_size = {}'.format(img_size))

        # Grab the outer four detected corners
        src = np.float32([ [405.5, 319], [650.5, 114.], [725.80072021, 330.18017578], [450.37277222, 517.24407959] ])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result
        # again, not exact, but close enough for our purposes

        dst = np.float32([[offset, offset], [img_size[0]-offset, offset],
                                     [img_size[0]-offset, img_size[1]-offset],
                                     [offset, img_size[1]-offset]])

        # Given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        # Warp the image using OpenCV warpPerspective()
        warped = cv2.warpPerspective(undist, M, img_size)

    # Return the resulting image and matrix
    return warped, M

top_down, perspective_M = corners_unwarp(img, nx, ny, mtx, dist)
cv2.imwrite('test_out/test_top_down.jpg', top_down)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=36)
ax2.imshow(top_down)
ax2.set_title('Undistorted and Warped Image', fontsize=36)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)




In [ ]:
import cv2
from glob import iglob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pickle

%matplotlib inline


img = mpimg.imread('./camera_cal/calibration13.jpg')
plt.imshow(img)
plt.plot(405.5, 319., '*')
plt.plot(650.5, 114, '*')
plt.plot(725.80072021, 330.18017578, '*')
plt.plot(450.37277222, 517.24407959, '*')


In [ ]:
from calibrate_camera import get_calibration_data, undistort_image


LINE_THICKNESS = 4
LINE_COLOR = [255, 0, 0]
TOP = 450
BOTTOM = 670

# Source points:
left_top = (592, TOP)
left_bottom = (260, BOTTOM)
right_top = (686, TOP)
right_bottom = (1050, BOTTOM)

src = np.float32([
    [left_top[0], left_top[1]],
    [right_top[0], right_top[1]],
    [right_bottom[0], right_bottom[1]],
    [left_bottom[0], left_bottom[1]]
])

dst_left = 320
dst_right = 960
dst = np.float32([
    [dst_left, 0],
    [dst_right, 0],
    [dst_right, 720],
    [dst_left, 720]
])

calibration = get_calibration_data()
matrix = calibration['mtx']
distortion_coefficients = calibration['dist']
orig_img = cv2.cvtColor(cv2.imread('./test_images/straight_lines1.jpg'), cv2.COLOR_BGR2RGB)
img = undistort_image(orig_img, matrix, distortion_coefficients)
img_size = (img.shape[1], img.shape[0])

img_copy = np.copy(img)
cv2.line(img_copy, left_bottom, left_top, LINE_COLOR, LINE_THICKNESS)
cv2.line(img_copy, right_bottom, right_top, LINE_COLOR, LINE_THICKNESS)
cv2.line(img_copy, left_top, right_top, LINE_COLOR, LINE_THICKNESS)
cv2.line(img_copy, left_bottom, right_bottom, LINE_COLOR, LINE_THICKNESS)

matrix = cv2.getPerspectiveTransform(src, dst)
warped = cv2.warpPerspective(img, matrix, img_size)
warped_lines = np.copy(warped)
cv2.line(warped_lines, (dst_left, 0), (dst_left, 720), LINE_COLOR, LINE_THICKNESS)
cv2.line(warped_lines, (dst_right, 0), (dst_right, 720), LINE_COLOR, LINE_THICKNESS)

plt.figure(figsize=(24, 15))
# f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
# plt.tight_layout()
plt.subplot(321)
plt.title('Original Image', fontsize=24)
plt.imshow(orig_img)

plt.subplot(322)
plt.title('Undistorted', fontsize=24)
plt.imshow(img)
# ax3.imshow(threshed, cmap='gray')
# ax3.set_title('Thresholded Gradient [Sobel]', fontsize=24)
# plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.subplot(323)
plt.title('Region of Interest', fontsize=24)
plt.imshow(img_copy)

plt.subplot(324)
plt.title('Warped', fontsize=24)
plt.imshow(warped_lines)


#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Then I did some other stuff and fit my lane lines with a 2nd order polynomial kinda like this:

![alt text][image5]

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


combined_binary_img_size = (combined_binary.shape[1], combined_binary.shape[0])
matrix = cv2.getPerspectiveTransform(src, dst)
binary_warped = cv2.warpPerspective(combined_binary, matrix, img_size)


# Assuming you have created a warped binary image called "binary_warped"
# Take a histogram of the bottom half of the image
histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

plt.figure(figsize=(5,5))
plt.subplot(211)
plt.imshow(binary_warped, cmap='gray')
plt.subplot(212)
plt.plot(histogram)

In [ ]:
# Create an output image to draw on and  visualize the result
out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
# Find the peak of the left and right halves of the histogram
# These will be the starting point for the left and right lines
midpoint = np.int(histogram.shape[0]/2)
leftx_base = np.argmax(histogram[:midpoint])
rightx_base = np.argmax(histogram[midpoint:]) + midpoint

# Choose the number of sliding windows
nwindows = 9
# Set height of windows
window_height = np.int(binary_warped.shape[0]/nwindows)
# Identify the x and y positions of all nonzero pixels in the image
nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])
# Current positions to be updated for each window
leftx_current = leftx_base
rightx_current = rightx_base
# Set the width of the windows +/- margin
margin = 100
# Set minimum number of pixels found to recenter window
minpix = 50
# Create empty lists to receive left and right lane pixel indices
left_lane_inds = []
right_lane_inds = []

# Step through the windows one by one
for window in range(nwindows):
    # Identify window boundaries in x and y (and right and left)
    win_y_low = binary_warped.shape[0] - (window+1)*window_height
    win_y_high = binary_warped.shape[0] - window*window_height
    win_xleft_low = leftx_current - margin
    win_xleft_high = leftx_current + margin
    win_xright_low = rightx_current - margin
    win_xright_high = rightx_current + margin
    # Draw the windows on the visualization image
    cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
    cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
    # Identify the nonzero pixels in x and y within the window
    good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    # Append these indices to the lists
    left_lane_inds.append(good_left_inds)
    right_lane_inds.append(good_right_inds)
    # If you found > minpix pixels, recenter next window on their mean position
    if len(good_left_inds) > minpix:
        leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
    if len(good_right_inds) > minpix:        
        rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

# Concatenate the arrays of indices
left_lane_inds = np.concatenate(left_lane_inds)
right_lane_inds = np.concatenate(right_lane_inds)

# Extract left and right line pixel positions
leftx = nonzerox[left_lane_inds]
lefty = nonzeroy[left_lane_inds] 
rightx = nonzerox[right_lane_inds]
righty = nonzeroy[right_lane_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

# Generate x and y values for plotting
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
plt.figure(figsize=(9,9))
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0);

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in lines # through # in my code in `my_other_file.py`

In [ ]:
# Generate some fake data to represent lane-line pixels
# ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
# quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
# For each y position generate random x position within +/-50 pix
# of the line base position in each case (x=200 for left, and x=900 for right)
# leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
#                               for y in ploty])
# rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
#                                 for y in ploty])

# leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
# rightx = rightx[::-1]  # Reverse to match top-to-bottom in y


# # Fit a second order polynomial to pixel positions in each fake lane line
# left_fit = np.polyfit(ploty, leftx, 2)
# left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
# right_fit = np.polyfit(ploty, rightx, 2)
# right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# # Plot up the fake data
mark_size = 3
plt.plot(leftx, lefty, 'o', color='red', markersize=mark_size)
plt.plot(rightx, righty, 'o', color='blue', markersize=mark_size)
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images

# Define y-value where we want radius of curvature
# I'll choose the maximum y-value, corresponding to the bottom of the image
y_eval = np.max(ploty)
left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
print(left_curverad, right_curverad)
# Example values: 1926.74 1908.48

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)

# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

# Now our radius of curvature is in meters
print(left_curverad, 'm', right_curverad, 'm')
# Example values: 632.1 m    626.2 m

In [ ]:
# Create an image to draw the lines on
warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (255, 255, 0))
cv2.putText(
    undistorted,
    'Curvature Radius = 651m',
    (50, 100),
    0,
    2,
    (255, 255, 0),
    2,
    cv2.LINE_AA
)
cv2.putText(
    undistorted,
    'Vehicle is {}m {} of center'.format(0.5, 'left'),
    (50, 160),
    cv2.FONT_HERSHEY_SIMPLEX,
    2,
    (255, 255, 0),
    2,
    cv2.LINE_AA
)

# Warp the blank back to original image space using inverse perspective matrix (Minv)
matrix_inverse = cv2.getPerspectiveTransform(dst, src)
newwarp = cv2.warpPerspective(color_warp, matrix_inverse, combined_binary_img_size) 
# Combine the result with the original image
result = cv2.addWeighted(undistorted, 1, newwarp, 0.3, 0)
plt.figure(figsize=(15, 9))
plt.imshow(result)


#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines # through # in my code in `yet_another_file.py` in the function `map_lane()`.  Here is an example of my result on a test image:

![alt text][image6]

---

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./project_video.mp4)

---

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  